In [1]:
import os
os.chdir('../../')

In [2]:
import numpy as np
import pandas as pd

from scr.util import *
from scr.feature_engineering.engineering import *

In [3]:
df_train = pd.read_csv('data/feature_engineered/null_cat/train_null_cat.csv')
df_test = pd.read_csv('data/feature_engineered/null_cat/test_null_cat.csv')

In [5]:
display(df_train.iloc[:5, 21:])
display(df_test.iloc[:5, 20:])

,TypeofContactNULL,Motivation,EconomicPower,Child01,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,IsFamily,Monetary,MonetarySeg,FreaqencySeg,ContractRate_FM,ContractRate_G1,ContractRate_G2
0,0,14,32.356270,0,48,2.351729,9.572861,16.427030,1.791759,2.791759,0,1.0,0,22.297944,3,1,0.141921,0.244444,0.500000
1,0,14,50.350356,0,8,2.375794,8.197046,18.461797,1.098612,2.098612,1,1.0,0,14.183467,2,0,0.135781,0.108844,0.250000
2,0,3,37.607704,0,24,1.875412,7.597516,16.547390,1.609438,2.609438,2,1.0,1,20.175755,3,0,0.146226,0.212389,0.171569
3,0,3,25.394244,0,8,2.147262,9.301921,14.855633,0.693147,2.539721,3,0.8,0,8.800974,1,0,0.180328,0.032258,0.012048
4,0,3,12.462403,0,48,1.731967,11.868956,12.337779,1.609438,3.914157,4,0.8,0,20.057464,3,0,0.146226,0.084746,0.056604


,TypeofContactNULL,Motivation,EconomicPower,Child01,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,TravelCost,EconomicSegment,PackageMatch,IsFamily,Monetary,MonetarySeg,FreaqencySeg,ContractRate_FM,ContractRate_G1,ContractRate_G2
0,0,4,34.102241,0,0,1.801646,8.773409,16.526471,2.079442,3.079442,10,0.6,1,27.274468,3,1,0.141921,0.000000,0.000000
1,0,14,25.223082,0,8,1.823531,8.819260,14.755503,1.609438,2.609438,3,1.0,1,20.297492,3,0,0.146226,0.032258,0.200000
2,0,4,37.405322,0,24,1.430065,8.312294,15.086813,0.693147,1.693147,13,1.0,0,8.642465,1,0,0.180328,0.311594,0.285714
3,0,4,32.416696,0,24,1.489417,9.134037,17.829183,0.693147,2.539721,20,0.8,0,8.642131,1,0,0.180328,0.481818,0.451087
4,0,4,48.757169,0,48,1.985889,7.937675,15.127224,0.693147,1.693147,13,1.0,0,8.665614,1,0,0.180328,0.300000,0.247706


# 特徴量作成

## TypeOfContactNuLL
TypeOfContact が欠損値の場合を、明らかに学習させるための特徴量

In [47]:
df_train['TypeofContactNULL'] = df_train['TypeofContact'].apply(make_TypeOfContactNULL)
df_test['TypeofContactNULL'] = df_test['TypeofContact'].apply(make_TypeOfContactNULL)

## Motivation
顧客の旅行に対する意欲を表す特徴量

In [48]:
df_train['Motivation'] = df_train['NumberOfPersonVisiting'] * df_train['NumberOfFollowups'] + (df_train['Passport'] * 10)
df_test['Motivation'] = df_test['NumberOfPersonVisiting'] * df_test['NumberOfFollowups'] + (df_test['Passport'] * 10)

## EconomicPower
顧客の経済力を表す特徴量

In [49]:
df_train['EconomicPower'] = df_train['MonthlyIncome'] * (4 - df_train['CityTier']) * df_train['Gender'].apply(make_motivation_gender)
df_test['EconomicPower'] = df_test['MonthlyIncome'] * (4 - df_test['CityTier']) * df_test['Gender'].apply(make_motivation_gender)

## Child01
顧客に子どもがいるかどうかを表す特徴量

In [50]:
df_train['Child01'] = df_train['Child'].apply(make_child01)
df_test['Child01'] = df_test['Child'].apply(make_child01)

## TripEasier
旅行の行きやすさを表す特徴量

In [51]:
df_train['TripEasier'] = (5 - df_train['NumberOfPersonVisiting']) * df_train['ProductPitched'].apply(make_TripEasier_ProductPitched) * df_train['Marry'].apply(make_TripEasier_Marry) * df_train['Child'].apply(make_TripEasier_Child)
df_test['TripEasier'] = (5 - df_test['NumberOfPersonVisiting']) * df_test['ProductPitched'].apply(make_TripEasier_ProductPitched) * df_test['Marry'].apply(make_TripEasier_Marry) * df_test['Child'].apply(make_TripEasier_Child)

## SalesPerformance
営業担当者の単位時間当たりのパフォーマンス

In [52]:
df_train['SalesPerformance'] = df_train['NumberOfFollowups'] * df_train['PitchSatisfactionScore'] / df_train['DurationOfPitch']
df_test['SalesPerformance'] = df_train['NumberOfFollowups'] * df_test['PitchSatisfactionScore'] / df_test['DurationOfPitch']

## LivingCost
生活コストを表す数値

In [53]:
df_train['LivingCost'] = df_train['MonthlyIncome'] / (df_train['CityTier'].apply(make_LivingCost_CityTier) * (1 + df_train['Child'].apply(make_LivingCost_Child) + df_train['Marry'].apply(make_LivingCost_Marry) + df_train['Car'].apply(make_LivingCost_Car)))
df_test['LivingCost'] = df_test['MonthlyIncome'] / (df_test['CityTier'].apply(make_LivingCost_CityTier) * (1 + df_test['Child'].apply(make_LivingCost_Child) + df_test['Marry'].apply(make_LivingCost_Marry) + df_test['Car'].apply(make_LivingCost_Car)))

## EconomicStability
経済安定性指標

In [54]:
df_train['EconomicStability'] = df_train['MonthlyIncome'] * df_train['Occupation'].apply(make_EconomicStability_Occupation) * df_train['Designation'].apply(make_EconomicStability_Disignation)
df_test['EconomicStability'] = df_test['MonthlyIncome'] * df_test['Occupation'].apply(make_EconomicStability_Occupation) * df_test['Designation'].apply(make_EconomicStability_Disignation)

## NumberOfTrips_log

In [56]:
df_train['NumberOfTrips_log'] = np.log1p(df_train['NumberOfTrips'])
df_test['NumberOfTrips_log'] = np.log1p(df_test['NumberOfTrips'])

## TravelCost
総旅行費の推定

In [57]:
df_train['TravelCost'] = df_train['PreferredPropertyStar'].apply(make_TravelCost_PreferredPropertyStar) * (1 + df_train['NumberOfTrips_log'])
df_test['TravelCost'] = df_test['PreferredPropertyStar'].apply(make_TravelCost_PreferredPropertyStar) * (1 + df_test['NumberOfTrips_log'])

## EconomicSegment
経済力のセグメント

In [58]:
df_train['EconomicSegment'] = df_train['Occupation'] + df_train['Designation'] + df_train['CityTier'].astype(str)
df_test['EconomicSegment'] = df_test['Occupation'] + df_test['Designation'] + df_test['CityTier'].astype(str)

df_train = mapping_EconomicSegment(df_train)
df_test = mapping_EconomicSegment(df_test)

## PacakgeMatch
営業担当者がセールスした商品と、顧客の希望するホテルのランクがマッチしているかを表す数値

In [59]:
product_to_star = {
    'Basic': 3,
    'Standard': 3,
    'Deluxe': 4,
    'Super Deluxe': 5,
    'King': 5
}

# 製品と希望ホテルの星評価の適合度を計算
def calculate_fit(product, preferred_star):
    ideal_star = product_to_star[product]
    max_star_difference = 5  # 星評価の最大差（1星から5星）
    return max(0, 1 - abs(ideal_star - preferred_star) / max_star_difference)

# 新しい特徴量をデータフレームに追加
df_train['PackageMatch'] = df_train.apply(lambda row: calculate_fit(row['ProductPitched'], row['PreferredPropertyStar']), axis=1)
df_test['PackageMatch'] = df_test.apply(lambda row: calculate_fit(row['ProductPitched'], row['PreferredPropertyStar']), axis=1)

In [60]:
df_train[['PackageMatch', 'ProductPitched', 'PreferredPropertyStar']]

,PackageMatch,ProductPitched,PreferredPropertyStar
0,1.0,Basic,3.0
1,1.0,Standard,3.0
2,1.0,Basic,3.0
3,0.8,Standard,4.0
4,0.8,Basic,4.0
...,...,...,...
3484,1.0,Basic,3.0
3485,0.6,Basic,5.0
3486,1.0,Standard,3.0
3487,0.6,King,3.0


## IsFamily
家族か、そうでないか、2人以上ならば家族とする。

In [61]:
df_train['IsFamily'] = df_train.apply(lambda x: int(x['Marry'] == 'Married' or x['Child'] != '0_child'), axis=1)
df_test['IsFamily'] = df_test.apply(lambda x: int(x['Marry'] == 'Married' or x['Child'] != '0_child'), axis=1)

## FreaqencySeg
旅行頻度をセグメント化。RFMのF

In [29]:
df_train['FreaqencySeg'] = df_train['NumberOfTrips'].apply(make_TripFreaqency)
df_test['FreaqencySeg'] = df_test['NumberOfTrips'].apply(make_TripFreaqency)

## Monetary
旅行に費やした総費用。RFM分析のM

In [25]:
df_train['Monetary'] = df_train['MonthlyIncome'] * df_train['NumberOfTrips_log']
df_test['Monetary'] = df_test['MonthlyIncome'] * df_test['NumberOfTrips_log']

## MonetarySeg
Monetaryを区分で分割

In [51]:
df_train['MonetarySeg'] = df_train['Monetary'].apply(make_MonetarySeg)
df_test['MonetarySeg'] = df_test['Monetary'].apply(make_MonetarySeg)

## ContractRate_FM
RFMのFMの部分。Freaqency、Monetary毎の契約成功率

In [105]:
tmp = df_train.groupby(by=['FreaqencySeg', 'MonetarySeg'], as_index=False)['ProdTaken'].mean()
tmp = tmp.rename(columns={'ProdTaken': 'ContractRate_FM'})

In [59]:
df_train = df_train.merge(tmp, on=['FreaqencySeg', 'MonetarySeg', 'ContractRate_FM'], how='left')
df_test = df_test.merge(tmp, on=['FreaqencySeg', 'MonetarySeg', 'ContractRate_FM'], how='left')

## ContractRate_G1 (G: Group)
顧客セグメント毎の契約成功率。訓練データの集計データをテストデータに結合しているため、少し問題がある？

In [101]:
tmp = df_train.groupby(by=['AgeGroup', 'Gender', 'ProductPitched'], as_index=False)['ProdTaken'].mean()
tmp = tmp.rename(columns={'ProdTaken': 'ContractRate_G1'})

In [86]:
df_train = df_train.merge(tmp, on=['AgeGroup', 'Gender', 'ProductPitched'], how='left')
df_test = df_test.merge(tmp, on=['AgeGroup', 'Gender', 'ProductPitched'], how='left')

## ContractRate_G2
年代、性別、都市層、パスポートの有無ごとの契約成功率

In [112]:
tmp = df_train.groupby(by=['AgeGroup', 'Gender', 'ProductPitched', 'Passport'], as_index=False)['ProdTaken'].mean()
tmp = tmp.rename(columns={'ProdTaken': 'ContractRate_G2'})

In [113]:
df_train = df_train.merge(tmp, on=['AgeGroup', 'Gender', 'ProductPitched', 'Passport'], how='left')
df_test = df_test.merge(tmp, on=['AgeGroup', 'Gender', 'ProductPitched', 'Passport'], how='left')

## ContractRate_G3
上の分割方法から、性別を除いたもの

In [14]:
tmp = df_train.groupby(by=['AgeGroup', 'ProductPitched', 'Passport'], as_index=False)['ProdTaken'].mean()
tmp = tmp.rename(columns={'ProdTaken': 'ContractRate_G3'})

In [15]:
df_train = df_train.merge(tmp, on=['AgeGroup', 'ProductPitched', 'Passport'], how='left')
df_test = df_test.merge(tmp, on=['AgeGroup', 'ProductPitched', 'Passport'], how='left')

## ContractRate_G4
上の分割方法から、パスポートを除いたもの

In [16]:
tmp = df_train.groupby(by=['AgeGroup', 'ProductPitched'], as_index=False)['ProdTaken'].mean()
tmp = tmp.rename(columns={'ProdTaken': 'ContractRate_G4'})

In [17]:
df_train = df_train.merge(tmp, on=['AgeGroup', 'ProductPitched'], how='left')
df_test = df_test.merge(tmp, on=['AgeGroup', 'ProductPitched'], how='left')

## ContractRate_G5
役職、年代、パスポート

In [35]:
tmp = df_train.groupby(by=['Designation', 'AgeGroup', 'Passport'], as_index=False)['ProdTaken'].mean()
tmp = tmp.rename(columns={'ProdTaken': 'ContractRate_G5'})

In [36]:
df_train = df_train.merge(tmp, on=['Designation', 'AgeGroup', 'Passport'], how='left')
df_test = df_test.merge(tmp, on=['Designation', 'AgeGroup', 'Passport'], how='left')

## ContractRate_G6
上からパスポート抜き

In [37]:
tmp = df_train.groupby(by=['Designation', 'AgeGroup'], as_index=False)['ProdTaken'].mean()
tmp = tmp.rename(columns={'ProdTaken': 'ContractRate_G6'})

In [38]:
df_train = df_train.merge(tmp, on=['Designation', 'AgeGroup'], how='left')
df_test = df_test.merge(tmp, on=['Designation', 'AgeGroup'], how='left')

# 特徴量のデータ型

In [39]:
feature = [
    'Age',
    'TypeofContact',
    'CityTier',
    'DurationOfPitch',
    'Occupation',
    'Gender',
    'NumberOfPersonVisiting',
    'NumberOfFollowups',
    'ProductPitched',
    'PreferredPropertyStar',
    'NumberOfTrips',
    'Passport',
    'PitchSatisfactionScore',
    'Designation',
    'MonthlyIncome',
    'Marry',
    'Car',
    'Child',
    # 以下、作成特徴量
    'AgeGroup',
    'TypeofContactNULL',
    'Motivation',
    'EconomicPower',
    'Child01',
    'TripEasier',
    'SalesPerformance',
    'LivingCost',
    'EconomicStability',
    'TravelCost',
    'NumberOfTrips_log',
    'EconomicSegment',
    'PackageMatch',
    'IsFamily',
    'FreaqencySeg',
    'Monetary',
    'MonetarySeg',
    'ContractRate_FM',
    'ContractRate_G1',
    'ContractRate_G2',
    'ContractRate_G3',
    'ContractRate_G4',
    'ContractRate_G5',
    'ContractRate_G6'
]

In [40]:
float_columns = ['DurationOfPitch', 'MonthlyIncome', 'EconomicPower', 'SalesPerformance',
                'LivingCost', 'EconomicStability', 'TravelCost', 'NumberOfTrips_log',
                'PackageMatch', 'Monetary', 'ContractRate_FM', 'ContractRate_G1',
                'ContractRate_G2', 'ContractRate_G3', 'ContractRate_G4', 'ContractRate_G5',
                'ContractRate_G6']
int_columns = ['Age', 'CityTier', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'PreferredPropertyStar',
                'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'TypeofContactNULL',
                'Motivation', 'Child01', 'TripEasier', 'EconomicSegment',
                'IsFamily', 'FreaqencySeg', 'MonetarySeg']

In [41]:
for col in float_columns:
    df_train[col] = df_train[col].astype(float)
    df_test[col] = df_test[col].astype(float)

for col in int_columns:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

# 確認

In [42]:
display(df_train.iloc[:5, 20:])
display(df_test.iloc[:5, 19:])

,AgeGroup,TypeofContactNULL,Motivation,EconomicPower,Child01,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,...,Monetary,MonetarySeg,FreaqencySeg,ContractRate_FM,ContractRate_G1,ContractRate_G2,ContractRate_G3,ContractRate_G4,ContractRate_G5,ContractRate_G6
0,50s,0,14,32.356270,0,48,2.351729,9.572861,16.427030,1.791759,...,22.297944,3,1,0.141921,0.244444,0.500000,0.454545,0.192308,0.454545,0.197368
1,50s,0,14,50.350356,0,8,2.375794,8.197046,18.461797,1.098612,...,14.183467,2,0,0.135781,0.108844,0.250000,0.200000,0.097046,0.195122,0.095238
2,30s,0,3,37.607704,0,24,1.875412,7.597516,16.547390,1.609438,...,20.175755,3,0,0.146226,0.212389,0.171569,0.223382,0.261860,0.228512,0.266667
3,30s,0,3,25.394244,0,8,2.147262,9.301921,14.855633,0.693147,...,8.800974,1,0,0.180328,0.032258,0.012048,0.058091,0.068441,0.055556,0.065693
4,40s,0,3,12.462403,0,48,1.731967,11.868956,12.337779,1.609438,...,20.057464,3,0,0.146226,0.084746,0.056604,0.185185,0.229050,0.186747,0.232044


,AgeGroup,TypeofContactNULL,Motivation,EconomicPower,Child01,TripEasier,SalesPerformance,LivingCost,EconomicStability,NumberOfTrips_log,...,Monetary,MonetarySeg,FreaqencySeg,ContractRate_FM,ContractRate_G1,ContractRate_G2,ContractRate_G3,ContractRate_G4,ContractRate_G5,ContractRate_G6
0,40s,0,4,34.102241,0,0,1.801646,8.773409,16.526471,2.079442,...,27.274468,3,1,0.141921,0.000000,0.000000,0.000000,0.000000,0.044944,0.050000
1,30s,0,14,25.223082,0,8,1.823531,8.819260,14.755503,1.609438,...,20.297492,3,0,0.146226,0.032258,0.200000,0.181818,0.068441,0.181818,0.065693
2,20s,0,4,37.405322,0,24,1.430065,8.312294,15.086813,0.693147,...,8.642465,1,0,0.180328,0.311594,0.285714,0.383871,0.416201,0.391447,0.422096
3,20s,0,4,32.416696,0,24,1.489417,9.134037,17.829183,0.693147,...,8.642131,1,0,0.180328,0.481818,0.451087,0.383871,0.416201,0.108108,0.154762
4,40s,0,4,48.757169,0,48,1.985889,7.937675,15.127224,0.693147,...,8.665614,1,0,0.180328,0.300000,0.247706,0.185185,0.229050,0.186747,0.232044


# CSV出力

In [43]:
if 'test' in df_train.columns:
    df_train = df_train.drop(columns='test', axis=1)
    df_test = df_test.drop(columns='test', axis=1)

In [44]:
df_train.to_csv('data/feature_engineered/null_cat/train_null_cat.csv', index=False)
df_test.to_csv('data/feature_engineered/null_cat/test_null_cat.csv', index=False)